# step1 读取数据

In [1]:
# -*- coding:utf-8 -*-
# First, we'll import pandas, a data processing and CSV file I/O library
# import training data
import pandas as pd
t3 = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\links (table 3).csv')
t4 = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\routes (table 4).csv')
t5_train = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\trajectories(table 5)_training.csv')
t6_train = pd.read_csv("E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\volume(table 6)_training.csv")

In [2]:
# import testing data
t6_test_2h_ago = pd.read_csv("E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\testing_phase1\\volume(table 6)_test1.csv")
t5_test_2h_ago = pd.read_csv("E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\testing_phase1\\trajectories(table 5)_test1.csv")

In [3]:
t5_test_2h_ago.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,A,2,1026631,2016-10-18 06:00:14,110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...,27.54
1,C,3,1017312,2016-10-18 06:00:28,115#2016-10-18 06:00:28#9.35;102#2016-10-18 06...,139.53
2,A,2,1044671,2016-10-18 06:03:07,110#2016-10-18 06:03:07#7.22;123#2016-10-18 06...,26.01
3,A,3,1000017,2016-10-18 06:04:37,110#2016-10-18 06:04:37#7.39;123#2016-10-18 06...,78.40
4,A,2,1002850,2016-10-18 06:10:17,110#2016-10-18 06:10:17#14.33;123#2016-10-18 0...,71.00


# step2 按照20分钟的时间窗口重组数据

In [4]:
type(t5_train['starting_time'][0])

str

In [5]:
t5_train['starting_time'] = pd.to_datetime(t5_train['starting_time'], format="%Y-%m-%d %H:%M:%S")
type(t5_train['starting_time'][0])

pandas._libs.tslib.Timestamp

In [6]:
t5_train = t5_train.set_index(['starting_time'])
t5_train.head()

,intersection_id,tollgate_id,vehicle_id,travel_seq,travel_time
starting_time,,,,,
2016-07-19 00:14:24,B,3,1065642,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85
2016-07-19 00:35:56,B,3,1047198,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79
2016-07-19 00:37:15,B,1,1086390,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76
2016-07-19 00:37:59,A,2,1071181,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05
2016-07-19 00:56:21,B,1,1065807,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98


In [7]:
t5_train = t5_train.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
        .reset_index().rename(columns={'travel_time':'avg_travel_time'})
t5_train.head()

,starting_time,intersection_id,tollgate_id,avg_travel_time
0,2016-07-19 00:00:00,B,3,70.85
1,2016-07-19 00:20:00,A,2,58.05
2,2016-07-19 00:20:00,B,1,79.76
3,2016-07-19 00:20:00,B,3,148.79
4,2016-07-19 00:40:00,B,1,137.98


In [8]:
t5_test_2h_ago['starting_time'] = pd.to_datetime(t5_test_2h_ago['starting_time'], format="%Y-%m-%d %H:%M:%S")
t5_test_2h_ago = t5_test_2h_ago.set_index(['starting_time'])
t5_test_2h_ago = t5_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
                .reset_index().rename(columns={'travel_time':'avg_travel_time'})
t5_test_2h_ago.head()

,starting_time,intersection_id,tollgate_id,avg_travel_time
0,2016-10-18 06:00:00,A,2,41.097143
1,2016-10-18 06:00:00,A,3,69.480000
2,2016-10-18 06:00:00,B,1,57.820000
3,2016-10-18 06:00:00,B,3,54.750000
4,2016-10-18 06:00:00,C,1,145.350000


In [9]:
t6_train.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [10]:
t6_train['time'] = pd.to_datetime(t6_train['time'], format="%Y-%m-%d %H:%M:%S")
t6_train = t6_train.set_index(['time'])
t6_train = t6_train.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
            .reset_index().rename(columns={0:'volume'})

In [11]:
t6_train.head()

,time,tollgate_id,direction,volume
0,2016-09-19,1,0,13
1,2016-09-19,1,1,140
2,2016-09-19,2,0,2
3,2016-09-19,3,0,17
4,2016-09-19,3,1,181


In [12]:
t6_test_2h_ago['time'] = pd.to_datetime(t6_test_2h_ago['time'], format="%Y-%m-%d %H:%M:%S")
t6_test_2h_ago = t6_test_2h_ago.set_index(['time'])
t6_test_2h_ago = t6_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
            .reset_index().rename(columns={0:'volume'})

In [13]:
t6_test_2h_ago.head()

,time,tollgate_id,direction,volume
0,2016-10-18 06:00:00,1,0,13
1,2016-10-18 06:00:00,1,1,37
2,2016-10-18 06:00:00,2,0,24
3,2016-10-18 06:00:00,3,0,30
4,2016-10-18 06:00:00,3,1,23


# step3 生成测试数据

In [14]:
print t5_train.shape
print t6_train.shape

(25144, 4)
(10063, 4)


In [15]:
t5_train = t5_train.append(t5_test_2h_ago)
t6_train = t6_train.append(t6_test_2h_ago)

In [16]:
print t5_train.shape
print t6_train.shape

(25592, 4)
(10483, 4)


In [17]:
# 增加两个小时作为测试数据
t5_test_2h_ago['starting_time'] = t5_test_2h_ago['starting_time'] + pd.DateOffset(hours=2)
t6_test_2h_ago['time'] = t6_test_2h_ago['time'] + pd.DateOffset(hours=2)

In [18]:
# 构建测试数据
t5_test, t6_test = t5_test_2h_ago, t6_test_2h_ago
del t5_test_2h_ago, t6_test_2h_ago

In [19]:
# 移除标签, inplace表示是否更新？
t5_test.drop('avg_travel_time',axis=1,inplace=True)
t6_test.drop('volume',axis=1,inplace=True)

In [20]:
t5_test.head()

,starting_time,intersection_id,tollgate_id
0,2016-10-18 08:00:00,A,2
1,2016-10-18 08:00:00,A,3
2,2016-10-18 08:00:00,B,1
3,2016-10-18 08:00:00,B,3
4,2016-10-18 08:00:00,C,1


In [21]:
t6_test.head()

,time,tollgate_id,direction
0,2016-10-18 08:00:00,1,0
1,2016-10-18 08:00:00,1,1
2,2016-10-18 08:00:00,2,0
3,2016-10-18 08:00:00,3,0
4,2016-10-18 08:00:00,3,1


# step4 生成日期特征

In [22]:
t5_test['starting_time'].dt.date.loc[0]

datetime.date(2016, 10, 18)

In [23]:
# 中秋国庆假期
from datetime import datetime
start1 = datetime(2016, 9, 15)
end1 = datetime(2016, 9, 17)
start2 = datetime(2016, 10, 1)
end2 = datetime(2016, 10, 7)
# 增加开学日期
start3 = datetime(2016, 8, 29)
end3 = datetime(2016, 9, 2)
rng = pd.date_range(start1, end1).append(pd.date_range(start2, end2)).append(pd.date_range(start3, end3))

In [24]:
rng

DatetimeIndex(['2016-09-15', '2016-09-16', '2016-09-17', '2016-10-01',
               '2016-10-02', '2016-10-03', '2016-10-04', '2016-10-05',
               '2016-10-06', '2016-10-07', '2016-08-29', '2016-08-30',
               '2016-08-31', '2016-09-01', '2016-09-02'],
              dtype='datetime64[ns]', freq=None)

In [25]:
t5_test.loc[0]

starting_time      2016-10-18 08:00:00
intersection_id                      A
tollgate_id                          2
Name: 0, dtype: object

In [26]:
# 是否假期,星期,小时,在那个时间窗口
# 感觉hour 和 time没什么用，如果觉得某一个特征有用，可以直接进行特征重要性检测
def generateDataTime(X, label):
    # 小时,星期,时间窗口
    hour = pd.get_dummies(X[label].dt.hour, prefix='hour_')
    weekday = pd.get_dummies(X[label].dt.weekday_name)
    time = pd.get_dummies(X[label].dt.time)
    # 添加到X
    X = pd.concat([X,weekday,hour,time], axis=1)
    date = X[label].dt.date
    for i, row in X.iterrows():
        X.loc[i, "holiday"] = 0
        if date.loc[i] in rng: X.loc[i, "holiday"] = 1
    return X

In [27]:
t5_test = generateDataTime(t5_test, "starting_time")
t6_test = generateDataTime(t6_test, "time")
t5_train = generateDataTime(t5_train, "starting_time")
t6_train = generateDataTime(t6_train, "time")

In [28]:
t5_train.head()

,starting_time,intersection_id,tollgate_id,avg_travel_time,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,...,21:00:00,21:20:00,21:40:00,22:00:00,22:20:00,22:40:00,23:00:00,23:20:00,23:40:00,holiday
0,2016-07-19 00:00:00,B,3,70.85,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
1,2016-07-19 00:20:00,A,2,58.05,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
2,2016-07-19 00:20:00,B,1,79.76,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
3,2016-07-19 00:20:00,B,3,148.79,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
4,2016-07-19 00:40:00,B,1,137.98,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0


## step5 提取路径特征

In [29]:
t4 = pd.read_csv('E:\Competetion\KDD-cup-2017\KDD_OFFICIAL_DATA\dataSets\\training\\routes (table 4).csv')
t4.head()

,intersection_id,tollgate_id,link_seq
0,A,2,"110,123,107,108,120,117"
1,A,3,"110,123,107,108,119,114,118,122"
2,B,1,"105,100,111,103,116,101,121,106,113"
3,B,3,"105,100,111,103,122"
4,C,1,"115,102,109,104,112,111,103,116,101,121,106,113"


In [30]:
# 将110，123变成[110, 123]
def split(str): return str.split(',')
t4.link_seq = t4.link_seq.apply(split)
t4.head()

,intersection_id,tollgate_id,link_seq
0,A,2,"[110, 123, 107, 108, 120, 117]"
1,A,3,"[110, 123, 107, 108, 119, 114, 118, 122]"
2,B,1,"[105, 100, 111, 103, 116, 101, 121, 106, 113]"
3,B,3,"[105, 100, 111, 103, 122]"
4,C,1,"[115, 102, 109, 104, 112, 111, 103, 116, 101, ..."


In [31]:
# 将数据按各link展开
rows = []
_ = t4.apply(lambda row: [rows.append([row['intersection_id'], row['tollgate_id'], nn]) 
                         for nn in row.link_seq], axis=1)
col = ['intersection_id', 'tollgate_id', 'link_id']
t4_new = pd.DataFrame(rows, columns=col)

In [32]:
t4_new.head()

,intersection_id,tollgate_id,link_id
0,A,2,110
1,A,2,123
2,A,2,107
3,A,2,108
4,A,2,120


In [33]:
# 对表三进行处理
t3.head()

,link_id,length,width,lanes,in_top,out_top,lane_width
0,100,58,3,1,105,111,3
1,101,84,3,1,116,121,3
2,102,131,9,3,115,109,3
3,103,23,12,4,111,"122,116",3
4,104,293,9,3,109,112,3


In [34]:
# 判断路段是否为交叉入口或者出口
t3['cross_in'] = 0
t3['cross_out'] = 0
for i, row in t3.iterrows():
    if ',' in str(row['in_top']):
        t3.loc[i, 'cross_in'] = 1
    if ',' in str(row['out_top']):
        t3.loc[i, 'cross_out'] = 1

In [35]:
t3.head()

,link_id,length,width,lanes,in_top,out_top,lane_width,cross_in,cross_out
0,100,58,3,1,105,111,3,0,0
1,101,84,3,1,116,121,3,0,0
2,102,131,9,3,115,109,3,0,0
3,103,23,12,4,111,"122,116",3,0,1
4,104,293,9,3,109,112,3,0,0


In [36]:
# 将整数型数据转换成string类型
t3['link_id'] = t3['link_id'].astype(str)

In [37]:
t4_new['link_id'] = t4_new['link_id'].astype(str)

In [38]:
t4_new = pd.merge(t4_new, t3, on='link_id', how='left')
t4_new.drop(['in_top', 'out_top'], axis=1, inplace=True)

In [39]:
t4_new.head()

,intersection_id,tollgate_id,link_id,length,width,lanes,lane_width,cross_in,cross_out
0,A,2,110,109,9,3,3,0,0
1,A,2,123,59,9,3,3,0,0
2,A,2,107,34,9,3,3,0,0
3,A,2,108,40,9,3,3,0,1
4,A,2,120,6,3,1,3,0,0


In [40]:
# 对交叉出入口进行聚合
b1 = t4_new[['intersection_id', 'tollgate_id', 'cross_in']]\
        .groupby(['intersection_id', 'tollgate_id'])\
        .cross_in.sum().reset_index().rename(columns={'cross_in':'in_link_cross_conut'})
b2 = t4_new[['intersection_id', 'tollgate_id', 'cross_out']]\
        .groupby(['intersection_id', 'tollgate_id'])\
        .cross_out.sum().reset_index().rename(columns={'cross_out':'out_link_cross_count'})
road = pd.merge(b1,b2,on=['intersection_id', 'tollgate_id'],how='left')
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count
0,A,2,0,1
1,A,3,1,2
2,B,1,2,1
3,B,3,2,1
4,C,1,2,1


In [41]:
# 路程
b1 = t4_new[['intersection_id', 'tollgate_id', 'length']].groupby(['intersection_id', 'tollgate_id']).length.sum().reset_index()
road = pd.merge(road, b1, on=['intersection_id', 'tollgate_id'], how='left')
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count,length
0,A,2,0,1,384
1,A,3,1,2,852
2,B,1,2,1,821
3,B,3,2,1,477
4,C,1,2,1,1550


In [42]:
## 车道数1车道2车道3车道4车道的link数，后期考虑：占总路程的比率
# 各个路径的link总数
b1 = t4_new[['intersection_id', 'tollgate_id']]\
        .groupby(['intersection_id', 'tollgate_id']).size()\
        .reset_index().rename(columns={0:'link_count'})
road = pd.merge(road, b1, on=['intersection_id', 'tollgate_id'], how='left')
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count,length,link_count
0,A,2,0,1,384,6
1,A,3,1,2,852,8
2,B,1,2,1,821,9
3,B,3,2,1,477,5
4,C,1,2,1,1550,12


In [43]:
# 1,2,3,4车道道路长度
# 测试1
b1 = t4_new[t4_new.lanes==1][['intersection_id', 'tollgate_id', 'length']]\
        .groupby(['intersection_id', 'tollgate_id']).length.sum()\
        .reset_index().rename(columns={'length':'1_length'})
road = pd.merge(road, b1, on=['intersection_id', 'tollgate_id'],how='left')

In [44]:
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count,length,link_count,1_length
0,A,2,0,1,384,6,142
1,A,3,1,2,852,8,404
2,B,1,2,1,821,9,275
3,B,3,2,1,477,5,255
4,C,1,2,1,1550,12,217


In [45]:
# 一次处理2、3、4
b1 = t4_new[t4_new.lanes==2][['intersection_id', 'tollgate_id', 'length']]\
        .groupby(['intersection_id', 'tollgate_id']).length.sum()\
        .reset_index().rename(columns={'length':'2_length'})
road = pd.merge(road, b1, on=['intersection_id', 'tollgate_id'],how='left')
b1 = t4_new[t4_new.lanes==3][['intersection_id', 'tollgate_id', 'length']]\
        .groupby(['intersection_id', 'tollgate_id']).length.sum()\
        .reset_index().rename(columns={'length':'3_length'})
road = pd.merge(road, b1, on=['intersection_id', 'tollgate_id'],how='left')
b1 = t4_new[t4_new.lanes==4][['intersection_id', 'tollgate_id', 'length']]\
        .groupby(['intersection_id', 'tollgate_id']).length.sum()\
        .reset_index().rename(columns={'length':'4_length'})
road = pd.merge(road, b1, on=['intersection_id', 'tollgate_id'],how='left')

In [46]:
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count,length,link_count,1_length,2_length,3_length,4_length
0,A,2,0,1,384,6,142,NaN,242.0,NaN
1,A,3,1,2,852,8,404,NaN,448.0,NaN
2,B,1,2,1,821,9,275,222.0,180.0,144.0
3,B,3,2,1,477,5,255,78.0,NaN,144.0
4,C,1,2,1,1550,12,217,144.0,1045.0,144.0


In [47]:
## 1,2,3,4车道link数
b1 = t4_new[t4_new.lanes== 1][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'1_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
b1 = t4_new[t4_new.lanes== 2][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'2_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
b1 = t4_new[t4_new.lanes== 3][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'3_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
b1 = t4_new[t4_new.lanes== 4][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'4_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')

In [48]:
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count,length,link_count,1_length,2_length,3_length,4_length,1_count,2_count,3_count,4_count
0,A,2,0,1,384,6,142,NaN,242.0,NaN,2,NaN,4.0,NaN
1,A,3,1,2,852,8,404,NaN,448.0,NaN,3,NaN,5.0,NaN
2,B,1,2,1,821,9,275,222.0,180.0,144.0,4,2.0,1.0,2.0
3,B,3,2,1,477,5,255,78.0,NaN,144.0,2,1.0,NaN,2.0
4,C,1,2,1,1550,12,217,144.0,1045.0,144.0,3,1.0,6.0,2.0


In [49]:
road.fillna(0, inplace=True)

In [50]:
road.head()

,intersection_id,tollgate_id,in_link_cross_conut,out_link_cross_count,length,link_count,1_length,2_length,3_length,4_length,1_count,2_count,3_count,4_count
0,A,2,0,1,384,6,142,0.0,242.0,0.0,2,0.0,4.0,0.0
1,A,3,1,2,852,8,404,0.0,448.0,0.0,3,0.0,5.0,0.0
2,B,1,2,1,821,9,275,222.0,180.0,144.0,4,2.0,1.0,2.0
3,B,3,2,1,477,5,255,78.0,0.0,144.0,2,1.0,0.0,2.0
4,C,1,2,1,1550,12,217,144.0,1045.0,144.0,3,1.0,6.0,2.0


In [51]:
t5_test = pd.merge(t5_test, road, on=['intersection_id', 'tollgate_id'], how='left')
t5_train = pd.merge(t5_train, road, on=['intersection_id', 'tollgate_id'], how='left')

In [52]:
t5_train.head()

,starting_time,intersection_id,tollgate_id,avg_travel_time,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,...,length,link_count,1_length,2_length,3_length,4_length,1_count,2_count,3_count,4_count
0,2016-07-19 00:00:00,B,3,70.85,0,0,0,0,0,1,...,477,5,255,78.0,0.0,144.0,2,1.0,0.0,2.0
1,2016-07-19 00:20:00,A,2,58.05,0,0,0,0,0,1,...,384,6,142,0.0,242.0,0.0,2,0.0,4.0,0.0
2,2016-07-19 00:20:00,B,1,79.76,0,0,0,0,0,1,...,821,9,275,222.0,180.0,144.0,4,2.0,1.0,2.0
3,2016-07-19 00:20:00,B,3,148.79,0,0,0,0,0,1,...,477,5,255,78.0,0.0,144.0,2,1.0,0.0,2.0
4,2016-07-19 00:40:00,B,1,137.98,0,0,0,0,0,1,...,821,9,275,222.0,180.0,144.0,4,2.0,1.0,2.0


## Step6 按要求存入数据，保存csv

In [53]:
# 设计end
t5_test['end'] = t5_test['starting_time'] + pd.DateOffset(minutes=20)
t5_train['end'] = t5_train['starting_time'] + pd.DateOffset(minutes=20)
t6_test['end'] = t6_test['time'] + pd.DateOffset(minutes=20)
t6_train['end'] = t6_train['time'] + pd.DateOffset(minutes=20)

In [54]:
t6_train.head()

,time,tollgate_id,direction,volume,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,...,21:20:00,21:40:00,22:00:00,22:20:00,22:40:00,23:00:00,23:20:00,23:40:00,holiday,end
0,2016-09-19,1,0,13,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2016-09-19 00:20:00
1,2016-09-19,1,1,140,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2016-09-19 00:20:00
2,2016-09-19,2,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2016-09-19 00:20:00
3,2016-09-19,3,0,17,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2016-09-19 00:20:00
4,2016-09-19,3,1,181,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,2016-09-19 00:20:00


In [55]:
# http://stackoverflow.com/questions/30132282/datetime-to-string-with-series-in-python-pandas
def timewindow(X, start, end):
    s = X[start].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"))
    e = X[end].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"))
    X['time_window'] = '[' + s + ',' + e + ')'
    return X.drop([start, end], axis=1)

In [56]:
t5_test = timewindow(t5_test, 'starting_time', 'end')
t5_train = timewindow(t5_train, 'starting_time', 'end')
t6_test = timewindow(t6_test, 'time', 'end')
t6_train = timewindow(t6_train, 'time', 'end')

In [57]:
t5_train.columns.values

array(['intersection_id', 'tollgate_id', 'avg_travel_time', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'hour__0', 'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5',
       'hour__6', 'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11',
       'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16',
       'hour__17', 'hour__18', 'hour__19', 'hour__20', 'hour__21',
       'hour__22', 'hour__23', datetime.time(0, 0), datetime.time(0, 20),
       datetime.time(0, 40), datetime.time(1, 0), datetime.time(1, 20),
       datetime.time(1, 40), datetime.time(2, 0), datetime.time(2, 20),
       datetime.time(2, 40), datetime.time(3, 0), datetime.time(3, 20),
       datetime.time(3, 40), datetime.time(4, 0), datetime.time(4, 20),
       datetime.time(4, 40), datetime.time(5, 0), datetime.time(5, 20),
       datetime.time(5, 40), datetime.time(6, 0), datetime.time(6, 20),
       datetime.time(6, 40), datetime.time(7, 0), datetime.time(7, 

In [58]:
t5_train.head()

,intersection_id,tollgate_id,avg_travel_time,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,...,link_count,1_length,2_length,3_length,4_length,1_count,2_count,3_count,4_count,time_window
0,B,3,70.85,0,0,0,0,0,1,0,...,5,255,78.0,0.0,144.0,2,1.0,0.0,2.0,"[2016-07-19 00:00:00,2016-07-19 00:20:00)"
1,A,2,58.05,0,0,0,0,0,1,0,...,6,142,0.0,242.0,0.0,2,0.0,4.0,0.0,"[2016-07-19 00:20:00,2016-07-19 00:40:00)"
2,B,1,79.76,0,0,0,0,0,1,0,...,9,275,222.0,180.0,144.0,4,2.0,1.0,2.0,"[2016-07-19 00:20:00,2016-07-19 00:40:00)"
3,B,3,148.79,0,0,0,0,0,1,0,...,5,255,78.0,0.0,144.0,2,1.0,0.0,2.0,"[2016-07-19 00:20:00,2016-07-19 00:40:00)"
4,B,1,137.98,0,0,0,0,0,1,0,...,9,275,222.0,180.0,144.0,4,2.0,1.0,2.0,"[2016-07-19 00:40:00,2016-07-19 01:00:00)"


In [59]:
t5_test = t5_test.set_index(['intersection_id','tollgate_id','time_window'])
t5_train = t5_train.set_index(['intersection_id','tollgate_id','time_window'])
t6_test = t6_test.set_index(['tollgate_id','time_window', 'direction'])
t6_train = t6_train.set_index(['tollgate_id','time_window', 'direction'])

In [65]:
t5_train.head()

avg_travel_time  \
intersection_id tollgate_id time_window                                                  
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)            70.85   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)            58.05   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)            79.76   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)           148.79   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)           137.98   

                                                                       Friday  \
intersection_id tollgate_id time_window                                         
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)       0   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)       0   

                                                                       Monday  \
intersection_id tollgate_id time_window                                         
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)       0   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)       0   

                                                                       Saturday  \
intersection_id tollgate_id time_window                                           
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)         0   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)         0   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)         0   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)         0   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)         0   

                                                                       Sunday  \
intersection_id tollgate_id time_window                                         
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)       0   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)       0   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)       0   

                                                                       Thursday  \
intersection_id tollgate_id time_window                                           
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)         0   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)         0   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)         0   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)         0   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)         0   

                                                                       Tuesday  \
intersection_id tollgate_id time_window                                          
B               3           [2016-07-19 00:00:00,2016-07-19 00:20:00)        1   
A               2           [2016-07-19 00:20:00,2016-07-19 00:40:00)        1   
B               1           [2016-07-19 00:20:00,2016-07-19 00:40:00)        1   
                3           [2016-07-19 00:20:00,2016-07-19 00:40:00)        1   
                1           [2016-07-19 00:40:00,2016-07-19 01:00:00)        1  

In [60]:
t5_test.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_test_task1.csv')
t5_train.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_train_task1.csv')
t6_test.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_test_task2.csv')
t6_train.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_train_task2.csv')

## step7 修复问题
* 发现测试和训练数据时间维度不同
* 原因：pd.getdummy()是吧所有可能取值都作为一列，但是测试数据时间的取值范围要比训练集小，顾会少很多列
* 办法：补充这些列

In [69]:
t5_testcol,t5_traincol = list(t5_test.columns.values),list(t5_train.columns.values)
t6_testcol,t6_traincol = list(t6_test.columns.values),list(t6_train.columns.values)
mis_5 =  [x for x in t5_traincol  if x not in t5_testcol]
mis_6 =  [x for x in t6_traincol  if x not in t6_testcol]

for label in mis_5:
    t5_test[label] = 0
for label in mis_6:
    t6_test[label] = 0

In [70]:
# 调整各列顺序
t5_test = t5_test[t5_traincol]
t6_test = t6_test[t6_traincol]

In [71]:
t5_test.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_test_task1.csv')
t5_train.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_train_task1.csv')
t6_test.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_test_task2.csv')
t6_train.to_csv('E:\Competetion\KDDCUP2017_others\KDD-CUP-2017_0\\dataSets\\table3456_train_task2.csv')

**注意**
**部分出口的某个时间窗口是有缺失的，所以到时候需要补充进去**